<a href="https://colab.research.google.com/github/polarpolar914/MachineLearning-for-AI/blob/main/week2/3-making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

In [1]:
!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/earthquakes.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/example_data.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/parsed.csv

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/quakes.db

!wget https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/tsunamis.csv

!mkdir data

!mv *.csv data

!mv *.db data

--2024-09-18 23:10:15--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/earthquakes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3515656 (3.4M) [text/plain]
Saving to: ‘earthquakes.csv’

earthquakes.csv     100%[===================>]   3.35M  --.-KB/s    in 0.1s    

2024-09-18 23:10:15 (32.8 MB/s) - ‘earthquakes.csv’ saved [3515656/3515656]

--2024-09-18 23:10:15--  https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas-2nd-edition/master/ch_02/data/example_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.1

### Get Data from API

In [2]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1726701021000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-08-18&endtime=2024-09-17',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 9957}

Each element in the JSON array `features` is a row of data for our dataframe.

In [5]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 0.99,
  'place': '10 km ENE of Borrego Springs, CA',
  'time': 1726530863440,
  'updated': 1726610416368,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci40735615',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci40735615&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 15,
  'net': 'ci',
  'code': '40735615',
  'ids': ',ci40735615,',
  'sources': ',ci,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 33,
  'dmin': 0.06384,
  'rms': 0.19,
  'gap': 128,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.0 - 10 km ENE of Borrego Springs, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-116.2848333, 33.3018333, 8.87]},
 'id': 'ci40735615'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,0.99,"10 km ENE of Borrego Springs, CA",1726530863440,1726610416368,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40735615,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",33.0,0.063840,0.1900,128.00,ml,earthquake,"M 1.0 - 10 km ENE of Borrego Springs, CA"
1,1.40,"48 km E of Fort Bidwell, California",1726530812264,1726531909484,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00884767,",",nn,",",origin,phase-data,",4.0,0.544000,0.1706,213.18,ml,earthquake,"M 1.4 - 48 km E of Fort Bidwell, California"
2,2.90,"40 km NW of Toyah, Texas",1726530571521,1726576602048,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000necp,tx2024shal,",",us,tx,",",origin,phase-data,",30.0,0.000000,0.2000,63.00,ml,earthquake,"M 2.9 - 40 km NW of Toyah, Texas"
3,1.71,"6 km WNW of Cobb, CA",1726530241510,1726536738079,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75062841,",",nc,",",focal-mechanism,nearby-cities,origin,phase-da...",32.0,0.009456,0.0400,32.00,md,earthquake,"M 1.7 - 6 km WNW of Cobb, CA"
4,1.47,"7 km SE of Valle Vista, CA",1726530120600,1726605626700,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40735607,",",ci,",",focal-mechanism,nearby-cities,origin,phase-da...",58.0,0.078200,0.2000,29.00,ml,earthquake,"M 1.5 - 7 km SE of Valle Vista, CA"


### (Optional) Write Data to CSV

In [8]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>